In [2]:
!pip install PyTDC

  Using cached pytdc-1.1.12.tar.gz (151 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached accelerate-0.33.0-py3-none-any.whl.metadata (18 kB)
  Using cached dataclasses-0.6-py3-none-any.whl.metadata (3.0 kB)
  Using cached datasets-2.19.2-py3-none-any.whl.metadata (19 kB)
  Using cached evaluate-0.4.2-py3-none-any.whl.metadata (9.3 kB)
  Using cached fuzzywuzzy-0.18.0-py2.py3-none-any.whl.metadata (4.9 kB)
  Using cached huggingface_hub-0.28.0-py3-none-any.whl.metadata (13 kB)
  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using 

  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [33 lines of output]
      Partial import of sklearn during the build process.
      Traceback (most recent call last):
        File "C:\Users\mhpie\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
        File "C:\Users\mhpie\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\mhpie\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 149, in prepare_metadata_for_build_wheel
          return hook(metadata_directory, config_settings)
                 ^^^^^^^^^^^

In [3]:
!pip install scikit-learn


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from tdc.multi_pred import TrialOutcome
data = TrialOutcome(name = 'phase1') # 'phase2' / 'phase3'
split = data.get_split()

ModuleNotFoundError: No module named 'tdc'

In [6]:
split["train"]

NameError: name 'split' is not defined

In [5]:
import time
from datetime import datetime

def get_timestamp(date_str):
    try:
        # Cas où le jour est présent (ex: "November 2, 2018")
        date_obj = datetime.strptime(date_str, "%B %d, %Y")
    except ValueError:
        # Cas où seul le mois et l'année sont présents (ex: "July 2000")
        date_obj = datetime.strptime(date_str, "%B %Y")
        date_obj = date_obj.replace(day=1)  # Ajouter le 1er du mois

    return int(date_obj.timestamp())  # Convertir en timestamp (secondes)

In [6]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialisation des encodeurs
drug_encoder = TfidfVectorizer()
icd_encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
eligibility_encoder = TfidfVectorizer()
duration_scaler = StandardScaler()

def compute_duration(start_date, complete_date):
    """ Calcule la durée en gérant les erreurs """
    duration = []
    for i in range(len(complete_date)):
        try:
            duration.append(get_timestamp(complete_date[i]) - get_timestamp(start_date[i]))
        except:
            duration.append(10**20)  # Valeur extrême pour éviter les erreurs
    return np.array(duration).reshape(-1, 1)  # S'assurer que c'est bien une colonne

def fit_transform_data(data):
    """ Applique fit_transform sur les données d'entraînement """
    global drug_encoder, icd_encoder, eligibility_encoder, duration_scaler  # On garde les mêmes encodeurs pour le test
    
    # Durée normalisée
    duration = compute_duration(data["start_date"], data["complete_date"])
    duration = duration_scaler.fit_transform(duration)  # Normalisation

    # Encodage des features
    drug_molecules = drug_encoder.fit_transform(np.array(data["drug_molecules"])).toarray()
    icdcodes = icd_encoder.fit_transform(np.array(data["icdcodes"]).reshape(-1, 1))
    eligibility_criteria = eligibility_encoder.fit_transform(np.array(data["eligibility_criteria"])).toarray()

    # Concaténation des features
    X_train = np.concatenate([duration, drug_molecules, icdcodes, eligibility_criteria], axis=1)
    return X_train

def transform_data(data):
    """ Applique transform sur les données de test (sans réentraîner les encodeurs) """
    global drug_encoder, icd_encoder, eligibility_encoder, duration_scaler

    duration = compute_duration(data["start_date"], data["complete_date"])
    duration = duration_scaler.transform(duration)  # Normalisation

    drug_molecules = drug_encoder.transform(np.array(data["drug_molecules"])).toarray()
    icdcodes = icd_encoder.transform(np.array(data["icdcodes"]).reshape(-1, 1))
    eligibility_criteria = eligibility_encoder.transform(np.array(data["eligibility_criteria"])).toarray()

    X_test = np.concatenate([duration, drug_molecules, icdcodes, eligibility_criteria], axis=1)
    return X_test

In [7]:
X_train = fit_transform_data(split["train"])  # Entraînement
Y_train_string = split["train"]["Y"]
Y_train = []

for k in range(len(Y_train_string)):
    Y_train.append(int(Y_train_string[k]))

Y_train = np.array(Y_train)

In [8]:
X_train

array([[ 9.6148034 ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.10400629,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.10400629,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.10400629,  0.        ,  0.        , ...,  0.11414377,
         0.        ,  0.        ],
       [-0.10400629,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.10400629,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [9]:
X_test = transform_data(split["test"])  # Transformation des données de test sans réentraîner

Y_test_string = split["test"]["Y"]
Y_test = []
for k in range(len(Y_test_string)):
    Y_test.append(int(Y_test_string[k]))

Y_test = np.array(Y_test)

In [ ]:
# Model random lineaire
from sklearn.linear_model import LogisticRegression  # Or LinearRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Initialize the model (for logistic regression if Y is binary)
model = LogisticRegression()

model.fit(X_train, Y_train)

# Make predictions
Y_pred = model.predict(X_test)  # Assuming X_test and Y_test are available for evaluation

# Evaluate the model
print("Accuracy:", accuracy_score(Y_test, Y_pred))
print("Confusion Matrix:\n", confusion_matrix(Y_test, Y_pred))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix

# Paramètres à tester pour le RandomForest
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialiser le modèle
rf = RandomForestClassifier(random_state=42)

# Effectuer la recherche sur les hyperparamètres avec une validation croisée
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1)

# Entraînement du modèle sur les données d'entraînement
grid_search.fit(X_train, Y_train)

# Meilleur modèle trouvé
best_model = grid_search.best_estimator_

# Faire des prédictions
Y_pred = best_model.predict(X_test)

# Évaluation du modèle
print("Accuracy:", accuracy_score(Y_test, Y_pred))
print("Confusion Matrix:\n", confusion_matrix(Y_test, Y_pred))



In [1]:
def list_mol(df):
    dico={}
    def molecule_existante(df):
        for i in df:
            j=i.split('_')
            for mol in j:
                if mol in dico:
                    dico[mol]+=1
                else:
                    dico[mol]=1
    molecule_existante(list(df["icdcodes"]))
    mol_list=[]
    for mol in list(df["icdcodes"]):
        mol_mol_list=[]
        for list_mol in dico.keys():
            if  list_mol in mol:
                mol_mol_list.append(1)
            else:
                mol_mol_list.append(0)
        mol_list.append(mol_mol_list)
    return mol_list

len(list_mol(split["train"]))


NameError: name 'split' is not defined